In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Create DataFrame from provided dataset
data = {
    "Date": pd.date_range(start="2022-06-19", periods=31, freq='D'),
    "Pods": [3090, 3088, 3088, 3088, 3120, 2910, 1210, 3090, 3088, 3088, 3088, 3120, 
             2910, 1210, 3090, 3088, 3086, 3086, 3120, 2910, 1210, 3100, 3106, 3106, 
             3106, 3120, 2910, 1210, 3120, 3120, 3120]
}
df = pd.DataFrame(data)

In [3]:
# Convert Date to datetime format and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [4]:
# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

seq_length = 5  # Number of time steps per sequence
X, y = create_sequences(df_scaled, seq_length)

# Parameters to test
layers_list = [2, 3, 4, 5, 6, 7]
train_rates = [0.7, 0.8, 0.9]

# Store results
results = []

# Loop over different configurations
for layers in layers_list:
    for train_rate in train_rates:
        # Split into training and testing sets
        train_size = int(len(X) * train_rate)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Build LSTM Model with variable layers
        model = Sequential()
        model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(seq_length, 1)))
        
        for _ in range(layers - 2):  # Add extra LSTM layers based on configuration
            model.add(LSTM(64, activation='relu', return_sequences=True))
        
        model.add(LSTM(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1))

        # Compile model
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

        # Train model
        model.fit(X_train, y_train, epochs=200, batch_size=4, validation_data=(X_test, y_test), verbose=0)

        # Make predictions
        y_pred = model.predict(X_test)
        y_pred_rescaled = scaler.inverse_transform(y_pred)
        y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Evaluate model
        mse = mean_squared_error(y_test_rescaled, y_pred_rescaled) * 100
        rmse = np.sqrt(mse) * 100
        mape = mean_absolute_percentage_error(y_test_rescaled, y_pred_rescaled) * 100
        wmape = (np.sum(np.abs(y_test_rescaled - y_pred_rescaled)) / np.sum(y_test_rescaled)) * 100

        # Store results
        results.append([layers, train_rate, mse, rmse, mape, wmape])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=["Layers", "Train Rate", "MSE (%)", "RMSE (%)", "MAPE (%)", "wMAPE (%)"])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step


In [5]:
# Determine the best configuration based on lowest error rate
best_layer = results_df.groupby("Layers")[["MSE (%)", "RMSE (%)", "MAPE (%)", "wMAPE (%)"]].mean().idxmin()
best_eval_method = results_df.iloc[:, 2:].mean().idxmin()

# Summary of results
summary = f"""
Best Layer Configuration: {best_layer.to_dict()}
Best Evaluation Method: {best_eval_method}
"""



In [ ]:
# Display results
import ace_tools as tools
tools.display_dataframe_to_user(name="LSTM Model Comparison Results", dataframe=results_df)

# Print summary
print(summary)